<a href="https://colab.research.google.com/github/worldbank/dec-python-course/blob/main/2-advanced-topics/text-analysis/intro-text-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción al Análisis de Texto

Análisis de texto es el proceso de extraer información significativa a partir de datos textuales, revelando ideas que de otro modo permanecerían ocultas en grandes volumenes de texto. El termino "texto" aqui se refiere a cualquier conjunto de caracteres: podria desde libros enteros hasta una sola oracion o palabra.

Esta sesión y la siguiente son una **introducción** al análisis de texto. Cubriremos los siguientes temas:

1. Estructuracion de datos de texto no estructurados
    1. Reconocimiento de caracteres desde documentos en PDF
1. Limpieza y preparacion de datos de texto
    1. Expresiones regulares y patrones de caracteres en datos de texto  
    1. Preprocesamiento de datos textuales  
1. Analisis descriptivo de datos de textos
    1. Conteo de palabras
    1. Análisis de sentimiento  
1. Clasificación de textos

Veremos los dos primeros puntos en la sesion de hoy y los dos ultimos puntos manana.

Esta sesión asume conocimientos previos de Python y Pandas, así como cierto conocimiento de visualización de datos usando seaborn. Todo lo que cubrimos en las tres primeras sesiones de este taller es suficiente base para continuar con analisis de textos.

Usaremos las siguientes bibliotecas en este notebook:

- **pdfminer** para "leer" archivos PDF y extraer su contenido en textos
- **pandas** para operaciones con dataframes  
- **re** para expresiones regulares  
- **spacy** para procesamiento de datos textuales

# 1. Estructuracion de datos de texto no estructurados

Los datos de en volumenes raramente tienen una estructura predeterminada. En muchos casos, estos vienen de archivos individuales que contienen textos. Por ejemplo, estos pueden ser un folder con decenas, cientos o miles de archivos en formato PDF, Word o `.txt`.

Dar una estructura a estos archivos requiere que evaluemos cual es la mejor forma que una tabla de datos para cierto volumen de textos puede adquirir. Por ejemplo, al trabajar con un folder con cientos de archivos PDF de texto, podemos estructurar la tabla para que tenga una fila por documento, una fila fila por parrafo, una fila por oracion o incluso una fila por palabra.

Para el siguiente ejemplo que usaremos en la primera mitad de esta sesion, partiremos de un volumen de documentos publicos del Banco Mundial. Estos documentos corresponden a algunos de los **reportes publicos que el Banco Mundial ha producido en Espanol sobre Republica Dominicana desde 2010 a 2024**.

Estos documentos estan en la carpeta `docs/` y son todos archivos en PDF. La imagen debajo es una captura de pantalla del contenido de la carpeta.

<img src="img/docs.png" width=800 />

## 1.1 Reconocimiento optico de caracteres desde documentos en PDF

El reconocimiento optico de caracteres (*Optical Character Recognition*--OCR por sus siglas en ingles) es una operacion comun en analisis de textos. Consiste en transformar textos que estan en algun documento (PDF or Word, por ejemplo) o en una imagen en un formato para texto que pueda ser operado por un lenguaje de programacion, por ejemplo en una string en Python.

En este ejemplo, usaremos el paquete `pdfminer` para reconocer los caracteres de estos archivos PDF. Ten en cuenta que tambien es posible reconocer textos de archivos en Word o de imagenes a strings en Python.

- [Mira aqui](https://github.com/microsoft/Simplify-Docx) un ejemplo (en ingles) para transformar archivos de Word a texto en Python
- [Aqui](https://medium.com/do-it-with-code/extract-text-from-images-using-python-ocr-dc7092adf9a8) un ejemplo (tambien en ingles) para transformar imagenes con textos a strings en Python

En general, existen muchas soluciones con metodos que logran resultados aceptables o buenos para reconocer caracteres de documentos o caracteres producidos con computadoras. Sin embargo, **reconocer escritura a mano**  es un proceso mucho mas complicado para el cual no existen soluciones gratuitas predeterminadas. El ejemplo que veremos en esta sesion y los dos links del parrafo anterior posiblemente no logren buenos resultados para caracteres escritos a mano.

Ahora continuaremos importando los paquetes necesarios para "leer" un PDF a texto en Python:

In [ ]:
# Instala PDF miner si no lo tienes
# !pip install pdfminer

# Modulos de pdfminer para leer PDF
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

# Paquetes para trabajar con directorios
import os
import io

Empezaremos creando una lista con la ubicacion de todos los documentos que queremos leer desde PDF. Estos estan en la carpeta `doc/`. Para esto, vamos a usar el paquete `os` que nos permite interactuar con folderes para trabajar con archivos.

In [ ]:
# Folder cuyos archivos queremos incluir en la lista
folder = 'docs/'

# Definiendo una lista vacia para agregar la ruta de los archivos PDF
docs = []

El siguiente loop explora todos los archivos en `folder` y anade a la lista `docs` aquellos que tienen la extension `.pdf`:

In [ ]:
# Bucle a traves de los archivos en "folder"
for archivo in os.listdir(folder):
    if archivo.endswith(".pdf"):             # si el archivo es un PDF, continuamos
        doc = os.path.join(folder, archivo)  # os.path.join une un nombre de carpeta y archivo para dar una ruta completa
        print(f'Documento: {doc}')
        docs.append(doc)                     # anadimos el archivo a la lista docs

In [ ]:
docs

In [ ]:
len(docs)

`docs` es ahora una lista con las rutas a los documentos PDF. Ahora vamos a iterar a traves de la lista para leerlos usando la funcion `texto_PDF()` que definiremos en el siguiente bloque.

Esta funcion es bastante complicada, pero funciona bien para casi todos los casos en que cualquier usuario tendria que leer un PDF. No es necesario entender todo lo que contiene la funcion dado que algunos de estos comandos son bastante especializados. Para nuestro uso, no la modificaremos y la vamos a utilizar tal como esta.

In [ ]:
def texto_PDF(pdfFile):
    
    # Basado en codigo de http://stackoverflow.com/a/20905381/4955164
    # El ejemplo usa encoding UTF-8. Esto se puede cambiar a otros encodings
    # para textos con caracteres inusuales
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams) #, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    
    return returnedString

Es muy importante notar que el input de `texto_pdf()` **no es la ruta del archivo PDF sino la lectura (en bytes) del archivo**. Para obtener la lectura en bytes, tenemos que abrir los archivos primero. Para esto usaremos una funcion muy frecuente para abrir archivos en Python: `open()` combinado con la palabra clave `with`:

In [ ]:
print(docs[0])

In [ ]:
# Abriremos el primer archivo en docs como ejemplo y lo usaremos con texto_PDF:
ruta_documento = docs[0]
with open(ruta_documento, 'rb') as f:
    texto = texto_PDF(f)

Usaremos `print()` para visualizar el resultado:

In [ ]:
print(texto)

El resultado no es perfecto, pero funcionaria bastante bien para realizar analisis de textos. Ahora continuaremos con procesar todos los documentos en la lista `docs`. Este proceso podria tomar algo de tiempo en terminarse.

In [ ]:
textos_completos = []

for doc in docs:
    
    with open(doc, 'rb') as f:
        
        print(f'Leyendo documento {doc}...')
        texto = texto_PDF(f)
        textos_completos.append(texto)
        print('\tFinalizado')

In [ ]:
len(textos_completos)

`textos_completos` ahora tiene los textos enteros de cada documento PDF listado en `docs`. Con esto, podemos seguir dando estructura a los textos en un dataframe de Pandas.

In [ ]:
import pandas as pd

In [ ]:
docs = docs[:-1]

In [ ]:
df_textos = pd.DataFrame()

In [ ]:
df_textos['archivos'] = docs
df_textos['textos'] = textos_completos

In [ ]:
df_textos

Ahora nuestros textos ya tienen una estructura de datos! con esto, podemos anadir nuevas columnas a `df_textos` con caracteristicas sobre los archivos que hemos leido. Para este ejemplo, anadiremos una columna con el numero de caracteres y otra con una variable dummy marcando cuales de los textos contienen las palabras clave "IVA" e "impuestos".

### Numero de caracteres

Usaremos el metodo de pandas `.apply()`. Este metodo aplica una funcion a cada uno de los elementos de una columna de un dataframe.

Recuerdas que mencionamos "vectorizacion" ayer? `.apply()` funciona de forma vectorizada, de modo que es mucho mas rapido que aplicar una funcion mediante un bucle.

In [ ]:
# Nueva columna
df_textos['n_caracteres'] = df_textos['textos'].apply(len)

In [ ]:
df_textos

### Palabras en el texto

Nuestra siguiente columna sera una *dummy* (un valor que es uno o cero, donde el valor uno indica la presencia de una caracteristica) indicando que textos contienen las palabras clave "IVA" o "impuestos".

Para esto, crearemos una funcion que toma un texto y verifica si las palabras clave estan en el. Luego usaremos `.apply()` para aplicar la funcion de forma vectorizada.

In [ ]:
def palabras_IVA_impuestos(texto):
    
    palabras = ['IVA', 'impuestos', 'Impuestos']
    
    for palabra in palabras:
        
        if palabra in texto:
            
            return 1
    
    return 0

In [ ]:
df_textos['mencion_IVA_impuestos'] = df_textos['textos'].apply(palabras_IVA_impuestos)

In [ ]:
df_textos

De esta forma podemos continuar agregando columnas al dataframe para proceder a analizar los textos. Esto es posible porque los datos ahora estan estructurados.

Por ahora, dejaremos de trabajar con este dataframe para continuar los ejemplos de esta y la proxima sesion utilizando textos mas pequenos en lugar de documentos enteros. Todas las operaciones que veremos a continuacion se pueden aplicar en los textos de `df_textos` o en textos largos.

# 2. Limpieza y preparacion de datos de textos

Para la siguiente parte, utilizaremos el archivo "muestra_ejercicio.xlsx". Este archivo contiene pequenos textos con el detalle de ventas reportadas por empresas en Republica Dominicana. Durante el resto de la sesion, vamos a trabajar con estos textos para modificarlos hasta que esten en una forma que sea util para analisis de datos de texto.

Comenzaremos con explorar los datos para entender que contienen.

### Exploracion de datos de texto

La exploracion de datos de texto no es muy distinta a la exploracion de datos numericos. Basicamente consiste en visualizar un dataframe con datos de texto para familiarizarnos con lo que contiene.

Empezaremos por cargar los datos. Por conveniencia, llamaremos a nuestro datadrame con los datos `df`:

In [ ]:
df = pd.read_excel('datos/muestra_ejercicio.xlsx', sheet_name='Muestra_clasificada')

In [ ]:
len(df)

In [ ]:
df.head()

Cada fila corresponde a un item reportado en el detalle de una boleta de venta. Para cada item, tenemos:
- El nombre reportado
- El tipo de ingreso por la venta
- Si es un bien o servicio
- La actividad del comercio emisor
- El nombre generico del producto
- El grupo del producto

Vamos a tabular algunas de estas columnas para tener una idea de cuan frecuentes son las categorias que se muestran:

In [ ]:
df['indicador_bien_servicio'].value_counts()

In [ ]:
df['GENERICO'].value_counts()

In [ ]:
df['CLASIFICACION'].value_counts()

In [ ]:
df['nombre_item'].value_counts()

La tabulacion de `nombre_item` indica varios puntos:
- La columna tiene muchas categorias unicas, que no se repiten
- La mayoria de caracteres estan en mayusculas
- La mayoria de categorias describen en una o dos palabras el producto
- Algunas tambien tienen informacion adicional en numeros o codigos

Ahora que ya conocemos mejor los datos, podemos comenzar a planear que hacer con ellos. Para esto, vamos a suponer que tenemos dos objetivos para estos datos:

- Hacer un analisis descriptivo de los textos en `nombre_item`.
    - Contar palabras
    - Hacer una nube de palabras
- Contruir un clasificador automatico que toma los textos en `nombre_item` y determina si el item es de la categoria alimento o no.

En concreto, realizaremos esto en la sesion de manana. Antes de eso, tenemos que preparar nuestros datos para que esten en una forma que resulta la mas adecuada para estos tipos de analisis.

## 2.1 Normalizacion de datos de texto

Para comprender por que la preparacion de textos es necesario, pensemos sobre lo siguiente?

- Nuestros textos están en un estado muy "crudo". ¿No deberíamos "limpiarlos" un poco antes de contar palabras? Por ejemplo, palabras en minusculas y mayusculas se cuentan por separado  
- Los textos en espanol (como en muchos otros idiomas) suelen repetir muchas palabras que no aportan mucho son muy informativas, como preposiciones, pronombres o contracciones. ¿Podemos eliminar algunas de ellas antes de contar palabras?  
- Por último, ¿no deberíamos contar en la misma categoría palabras que no son exactamente iguales pero tienen un significado muy similar? Por ejemplo:
    + diferentes conjugaciones del mismo verbo
    + formas singulares y plurales del mismo sustantivo

La respuesta a todas estas preguntas es **sí**. Lo haremos en la preparacion de los datos. La preparacion de datos en análisis de texto es **extremadamente importante**. Omitirlo puede darte resultados muy diferentes en tareas de análisis de texto.

El preprocesamiento y la preparacion puede incluir múltiples tareas. Aplicaremos las siguientes a nuestros textos:

- Convertir caracteres a minúsculas  
- Tokenización: transformar textos en listas de palabras  
- Eliminar palabras poco informativas. En lenguaje tecnico se conocen como *stop words*
- Lematización: transformar diferentes formas de palabras en una forma común que exprese un significado similar. Esto es útil para "normalizar" conjugaciones de verbos o formas 

Por suerte, existe un paquete de Python muy útil que podemos usar para esto: [spaCy](https://spacy.io/).  
spaCy pone a nuestra disposición modelos de NLP (procesamiento de lenguaje natural) que permiten tokenizar, lematizar y detectar *stop words* y caracteres que no son palabras (como dígitos o signos de puntuación), por lo que podemos transformar fácilmente un texto en una lista de palabras lematizadas "significativas" que podemos usar para contar palabras o construir clasificadores.

### Trabajando con spaCy

Primero necesitamos cargar spaCy.

In [ ]:
# Instala spaCy si no lo tienes:
#!pip install spacy

In [ ]:
import spacy

Now we need to **download** spaCy's NLP model. Uncomment the line below, run it only once, and then comment it out again to make sure you won't run it again accidentally.

Ahora necesitamos **descargar** el modelo de NLP de spaCy en espanol. spaCy tiene varios modelos en espanol. Puedes verlos [aqui](https://spacy.io/models/es%20). Los modelos de mayor tamano funcionan mejor, pero tambien requieren mas espacio de disco duro y poder computacional. Nostros trabajaremos con el modelo `es_core_news_md`, que no es muy pesado y logra buenos resultados.

In [ ]:
#!python -m spacy download es_core_news_md

Ahora **cargamos** el modelo para que esté disponible en este notebook:

In [ ]:
nlp = spacy.load('es_core_news_md')

Then, we'll build a function that:

1. Reads a text
1. Transforms it to lowercase
1. Loads it into the model
1. For each word, obtains the lemmatized versions of words that are not:
    - Stop words
    - Punctuation
    - Numbers
    - Spaces
1. Finally, the function returns a list of the lemmatized words

Luego, construiremos una función que:

1. Lew un texto  
1. Lo transforme a minúsculas  
1. Lo cargue al modelo de spaCy
1. Para cada palabra de mas de 2 caracteres, obtiene la versión lematizada de aquellas que no sean:  
    - *Stop words*  
    - Signos de puntuación  
    - Números  
    - Espacios  
1. Finalmente, la función devuelve una lista de las palabras lematizadas

In [ ]:
def normalizacion_tokenizacion(texto):
    
    texto = texto.lower() # minusculas
    doc = nlp(texto)      # cargando el texto al modelo de spaCy

    palabras_normalizadas = []
    for palabra in doc:
        if palabra.text != '\n' \
        and not palabra.is_stop \
        and not palabra.is_punct \
        and not palabra.like_num \
        and len(palabra.text.strip()) > 2:
            lema = str(palabra.lemma_)
            palabras_normalizadas.append(lema)
    
    return palabras_normalizadas

In [ ]:
texto = 'dominicanas'
texto = nlp(texto)
for palabra in texto:
    print(palabra.lemma_)

Para entender mejor lo que hace la función, veamos el resultado para un par de textos:

In [ ]:
texto = df['nombre_item'][1]
print(texto)

In [ ]:
normalizacion_tokenizacion(texto)

In [ ]:
texto = df['nombre_item'][74]
print(texto)

In [ ]:
normalizacion_tokenizacion(texto)

Notemos lo siguiente:

- El resultado es una lista, no un texto
- todas las palabras estan en minusculas
- Las preposiciones han sido eliminadas
- Los plurales han sido transformados a singular como parte de la lematizacion

Para nuestra sorpresa, tambien podemos ver que la palabra `3x5` no ha sido eliminada. Esto lo corregiremos posteriormente mediante el uso de expresiones regulares.

Por ahora, lo que nos queda es aplicar la funcion `normalizacion_tokenizacion()` de forma transversal a toda la columna `nombre_item`.

In [ ]:
df['nombre_item_norm'] = df['nombre_item'].apply(normalizacion_tokenizacionacion_tokenizacion)

In [ ]:
df

In [ ]:
df.head(10)

Visualizar estos resultados nos hace dar cuenta que hay mas codigos que no han sido eliminados en la normalizacion:

- `3x5`
- `rf-88`
- `ib1707`

Corregiremos esto mediante expresiones regulares.



## 2.2 Patrones en textos y expresiones regulares

Estos tres codigos parecen seguir patrones en los caracteres que contienen:

- `3x5`: un numero, seguido de una `x`, seguida de un numero
- `rf-88`: dos letras, seguidas de un guion, seguidas de dos numeros
- `ib1707`: dos letras, seguidas de cuatros numeros

Vamos a aprovechar estos patrones para eliminarlos del texto normalizado. Usaremos expresiones regulares para esto.

### Expresiones regulares

En programación, las expresiones regulares (*regex*) son secuencias de caracteres que coinciden con un patrón dentro de un texto. Un ejemplo simple:

In [ ]:
# re es el paquete en Python para trabajar con expresiones regulares.
import re

In [ ]:
texto = 'El número de DNI del contribuyente 1 es 30551. Nació el 1 de julio de 1976. El contribuyente 2 cuenta con DNI 71098'

# Patrón para capturar los ID en este texto: secuencias de cinco caracteres numéricos.
patron = '\d{5}'

# Capturando los DNI
dni = re.findall(patron, texto)
print(dni)

- `\d` es un codigo representa un número (0-9). Esto es equivalente a `[0-9]`
- `{5}` significa que el carácter anterior en el patrón se repite cinco veces
- Una variación de este patrón podría ser `\d{4}`, que podría usarse para capturar años. Esto habría devuelto una lista con `1996` en el ejemplo anterior

En regex, existe un codigo para casi todo. Algunos ejemplos:

- Codigos para caracteres:
    + `\d` --> dígitos (0-9)  
    + `\w` --> cualquier caracter de palabra (letras mayúsculas y minúsculas, dígitos numericos y el guion bajo `_`)  
    + `\n` --> saltos de línea  
    + `\s` --> caracteres de espacio en blanco, incluyendo saltos de línea  
    + `.` --> cualquier caracter excepto el salto de línea (\n)  

- Para repetición de caracteres:
    + `{a}` --> el carácter anterior, repetido exactamente "a" veces  
    + `{a,b}` --> el carácter anterior, repetido entre "a" y "b" veces  
    + `*` --> el carácter anterior, repetido cero o más veces  
    + `+` --> el carácter anterior, repetido una o más veces  

Regex puede detectar prácticamente cualquier patrón que podamos imaginar. Sin embargo, trabajar con expresiones regulares puede ser complejo al principio. En esta sesión hemos introducido el concepto de regex para que sepas que existe y que puede usarse para crear columnas en conjuntos de datos que contienen corpus de documentos.

No te preocupes si todavía no entendiste bien cómo funcionan los patrones. Si te interesa aprender más sobre regex, te recomendamos los siguientes recursos:

- Un buen tutorial sobre regex [aquí](https://regexone.com/)  
- Una excelente herramienta visualizadora de expresiones regulares está [aquí](https://jex.im/regulex/#!flags=&re=www%5C.%5Ba-zA-Z0-9-%5D%2B%5C.(%3F%3Acom%7Cnet%7Corg))

### Reemplazando información usando regex

El comando `re.findall()` se usa para extraer todas las menciones de una regex. Nosotros usaremos `re.match()`, que se usa para reemplazar verificar si un patron existe en un texto.

Nuestros patrones seran 3, de acuerdo a lo que ya hemos visto:

- un numero, seguido de una `x`, seguida de un numero
- dos letras minusculas, seguidas de un guion, seguidas de dos numeros
- dos letras minusculas, seguidas de cuatros numeros

In [ ]:
patron1 = '\dx\d'
patron2 = '[a-z]{2}-\d{2}'
patron3 = '[a-z]{2}\d{4}'

Y haremos una funcion que borra estos patrones en `nombre_item_norm`, que era la columna con las palabras normalizadas y en listas. Recuerda que el input de esta funcion debe ser una lista, no un texto:

In [ ]:
def eliminar_codigos(lista_palabras):
    
    nueva_lista = []
    
    for palabra in lista_palabras:
        
        if not re.match(patron1, palabra) and \
            not re.match(patron2, palabra) and \
            not re.match(patron3, palabra):
            
            nueva_lista.append(palabra)
    
    return nueva_lista

Para comprobar su funcionamiento, probaremos la funcion en la segunda observacion de `nombre_item_norm`:

In [ ]:
df['nombre_item_norm'][1]

In [ ]:
eliminar_codigos(df['nombre_item_norm'][1])

Ahora aplicaremos la funcion a toda la columna:

In [ ]:
df['nombre_item_final'] = df['nombre_item_norm'].apply(eliminar_codigos)

In [ ]:
df

Excelente! La columna resultante `nombre_item_final` contiene el texto preparado como lo vamos a necesitar para un analisis descriptivo y para construir un clasificador automatico.

# Comentarios finales

Antes de culminar, repasemos todos lo que hemos visto hoy:

- Lectura de archivos PDF
- Estructuracion de textos no estructurados en dataframes
- Generacion de nuevas columnas sobre atributos del texto
- Normalizacion de textos y tokenizacion
- Expresiones regulares

Todas estas tareas son tareas comunes en analisis de textos para preparar los datos. Manana veremos como utilizar estos datos preparados para realizar un analisis descriptivo y construir un clasificador.